In [1]:
import requests
import pandas as pd
import json
from pandas import json_normalize
import numpy as np
import pickle
from tensorflow.keras.layers import Dense, GRU , Flatten , LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
import keras

In [2]:
def read_addresses_and_labels(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        addresses = [line.strip() for line in lines]
    return addresses

file_path_cairo = 'Cairo.txt'
file_path_other = 'other.txt'

addresses_cairo = read_addresses_and_labels(file_path_cairo)
addresses_other = read_addresses_and_labels(file_path_other)

y_cairo = [1] * len(addresses_cairo)
y_other = [0] * len(addresses_other)

X = addresses_cairo + addresses_other
y = y_cairo + y_other


In [2]:
cities= pd.read_fwf("Cairo.txt",names=["Address"])
l = []
api_key = "YdwX1rPlKcS4pifCTqpFBYpf1a2ADcdXTSmmmHiQ2aM"
base_url = "https://discover.search.hereapi.com/v1/discover"
coords = "30.04427,31.23525"

for i in list(cities['Address'].values):
    url = f"{base_url}?apiKey={api_key}&at={coords}&q=[{i}]&languages=ar-US"
    try:
        response = requests.get(url=url)
        if response.status_code == 200:
            text = response.json()
            df = pd.json_normalize(text['items'], sep='_')
            df = df[['address_label', 'address_city']]
            l.append(df)
        else:
            print(f"Failed to fetch data for {i}. Status code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred for {i}: {e}")

if l:
    final_df = pd.concat(l, ignore_index=True)

else:
    print("No data fetched from the API.")


In [3]:
final_df['address_city'].unique()

array(['التبين', 'حلوان', 'مركز بلقاس', '15 مايو', 'طرة', 'المعادى',
       'القاهرة', 'إمبابة', 'العمرانية', 'بولاق الدكرور',
       'شبرا الخيمة قسم 2', 'العجوزة', 'مركز ابو طشت', 'المرج', nan,
       'الجيزة', 'القاهرة الجديدة', 'بدر'], dtype=object)

In [4]:
filtered_df = final_df[final_df['address_city'].isin(['التبين', 'حلوان', '15 مايو', 'طرة', 'المعادى', 'القاهرة', 'إمبابة', 'العمرانية', 'بولاق الدكرور', 'شبرا الخيمة قسم 2', 'العجوزة', 'المرج', 'الجيزة', 'القاهرة الجديدة', 'بدر'])]
filtered_df

,address_label,address_city
0,"التبين, مصر",التبين
1,"حلوان, مصر",حلوان
2,"المعصرة البلد, حلوان, مصر",حلوان
3,"المعصرة المحطة, حلوان, مصر",حلوان
5,"15 مايو, مصر",15 مايو
...,...,...
94,"القاهرة الجديدة, مصر",القاهرة الجديدة
95,"الشروق, القاهرة الجديدة, مصر",القاهرة الجديدة
96,"بدر, شارع الانشراح الصغير, العجوزة, الجيزة, مصر",العجوزة
97,"جهاز مدينة بدر, شارع ابن سينا, بدر, القاهرة, مصر",بدر


In [5]:
filtered_df['address_city'] = 'القاهرة'
filtered_df.reset_index(drop=True, inplace=True)
filtered_df

<ipython-input-5-10eb26b80390>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['address_city'] = 'القاهرة'


,address_label,address_city
0,"التبين, مصر",القاهرة
1,"حلوان, مصر",القاهرة
2,"المعصرة البلد, حلوان, مصر",القاهرة
3,"المعصرة المحطة, حلوان, مصر",القاهرة
4,"15 مايو, مصر",القاهرة
...,...,...
91,"القاهرة الجديدة, مصر",القاهرة
92,"الشروق, القاهرة الجديدة, مصر",القاهرة
93,"بدر, شارع الانشراح الصغير, العجوزة, الجيزة, مصر",القاهرة
94,"جهاز مدينة بدر, شارع ابن سينا, بدر, القاهرة, مصر",القاهرة


In [6]:
len(filtered_df)

96

In [7]:
cities= pd.read_fwf("other.txt",names=["Address"])
l = []
api_key = "YdwX1rPlKcS4pifCTqpFBYpf1a2ADcdXTSmmmHiQ2aM"
base_url = "https://discover.search.hereapi.com/v1/discover"
coords = "30.04427,31.23525"

for i in list(cities['Address'].values):
    url = f"{base_url}?apiKey={api_key}&at={coords}&q=[{i}]&languages=ar-US"
    try:
        response = requests.get(url=url)
        if response.status_code == 200:
            text = response.json()
            df = pd.json_normalize(text['items'], sep='_')
            df = df[['address_label', 'address_city']]
            l.append(df)
        else:
            print(f"Failed to fetch data for {i}. Status code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred for {i}: {e}")

if l:
    final_df_other = pd.concat(l, ignore_index=True)

else:
    print("No data fetched from the API.")


In [8]:
final_df_other['address_city'].unique()

array(['القاهرة', 'القاهرة الجديدة', 'תל אביב', 'الاسكندرية',
       'شبرا الخيمة قسم 1', 'مركز الابراهيمية', 'الجيزة', 'العجوزة',
       'الوراق', 'تمبدغة', 'مركز الجيزه', 'شبرا الخيمة قسم 2',
       'التل الكبير', 'بئر علي بن خليفة', 'حلوان', 'بولاق الدكرور',
       'إمبابة', 'مركز قليوب', 'الهرم', 'ابو رديس', 'الدقى', 'العمرانية',
       'الدوحة', 'راس الخيمة', 'المضيبي', 'مركز كوم حمادة',
       'مركز القناطر الخيريه', 'كرداسة', 'مركز الشهداء', 'مدينة منوف',
       'مدينة شبين الكوم', 'مركز قويسنا', 'مدينة سرس الليان', 'العامرية',
       'السادس من اكتوبر', 'المنصورة', 'عمان', 'قليوب', 'مركز الزقازيق',
       'الصف', 'طنطا', 'بورسعيد', 'جدة', 'مركز كفر الدوار', 'مركز إدكو',
       'مركز رشيد', 'دمنهور', 'مركز مطوبس', 'مركز دمنهور', 'دسوق',
       'مركز قلين', 'مركز كفر الزيات', 'كفر الشيخ', 'مركز كفر الشيخ',
       'مركز قطور', 'مركز تلا', 'كفرنجا الجديدة', 'طرة', 'بيروت', 'صيدا',
       'الزاوية', 'حولى', 'المنزلة', 'الباحة', 'مركز دكرنس', 'ميت غمر',
       'مركز جمصة', 'أجا', 'مر

In [9]:
want = ['الاسكندرية', 'مركز الابراهيمية', 'تمبدغة',
       'التل الكبير', 'بئر علي بن خليفة', 'مركز قليوب', 'ابو رديس', 'العمرانية',
       'الدوحة', 'المضيبي', 'مركز كوم حمادة',
       'مركز القناطر الخيريه', 'كرداسة', 'مركز الشهداء', 'مدينة منوف',
       'مدينة شبين الكوم', 'مركز قويسنا', 'مدينة سرس الليان', 'العامرية',
       'المنصورة', 'عمان', 'قليوب', 'مركز الزقازيق',
       'الصف', 'طنطا', 'بورسعيد', 'جدة', 'مركز كفر الدوار', 'مركز إدكو',
       'مركز رشيد', 'دمنهور', 'مركز مطوبس', 'مركز دمنهور', 'دسوق',
       'مركز قلين', 'مركز كفر الزيات', 'كفر الشيخ', 'مركز كفر الشيخ',
       'مركز قطور', 'مركز تلا', 'كفرنجا الجديدة', 'طرة', 'بيروت', 'صيدا',
       'حولى', 'المنزلة', 'الباحة', 'مركز دكرنس', 'ميت غمر',
       'مركز جمصة', 'أجا', 'مركز بلقاس', 'السنبلاوين', 'مركز شربين',
       'مركز طلخا', 'مركز بني عبيد', 'مركز منية النصر', 'مركز ميت سلسبيل',
       'مركز تمي الامديد', 'مركز ابو قرقاص', 'مركز نبروه']

In [10]:
filtered_df_other = final_df_other[final_df_other['address_city'].isin(want)]
filtered_df_other

,address_label,address_city
4,"الاسكندرية, مصر",الاسكندرية
7,"الإبراهيمية, مركز الابراهيمية, مصر",مركز الابراهيمية
8,"مركز الابراهيمية, مصر",مركز الابراهيمية
11,"البيطاش, شارع رجب الطيار, الاسكندرية, الاسكندر...",الاسكندرية
12,"البيطاش, شارع البيطاش, الاسكندرية, الاسكندرية,...",الاسكندرية
...,...,...
318,"التمديد, شارع حسني متولي, العمرانية, الجيزة, مصر",العمرانية
319,"مركز تمي الامديد, مصر",مركز تمي الامديد
320,"طريق تمى الامديد-السمبلاوين, السنبلاوين, الدقه...",السنبلاوين
322,"مركز نبروه, مصر",مركز نبروه


In [11]:
filtered_df_other['address_city'] = 'اخري'
filtered_df_other.reset_index(drop=True, inplace=True)
filtered_df_other

<ipython-input-11-fe1c9cbb5817>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_other['address_city'] = 'اخري'


,address_label,address_city
0,"الاسكندرية, مصر",اخري
1,"الإبراهيمية, مركز الابراهيمية, مصر",اخري
2,"مركز الابراهيمية, مصر",اخري
3,"البيطاش, شارع رجب الطيار, الاسكندرية, الاسكندر...",اخري
4,"البيطاش, شارع البيطاش, الاسكندرية, الاسكندرية,...",اخري
...,...,...
143,"التمديد, شارع حسني متولي, العمرانية, الجيزة, مصر",اخري
144,"مركز تمي الامديد, مصر",اخري
145,"طريق تمى الامديد-السمبلاوين, السنبلاوين, الدقه...",اخري
146,"مركز نبروه, مصر",اخري


In [12]:
df_f = pd.concat([filtered_df, filtered_df_other])
df_f.reset_index(drop=True, inplace=True)
df_f

,address_label,address_city
0,"التبين, مصر",القاهرة
1,"حلوان, مصر",القاهرة
2,"المعصرة البلد, حلوان, مصر",القاهرة
3,"المعصرة المحطة, حلوان, مصر",القاهرة
4,"15 مايو, مصر",القاهرة
...,...,...
239,"التمديد, شارع حسني متولي, العمرانية, الجيزة, مصر",اخري
240,"مركز تمي الامديد, مصر",اخري
241,"طريق تمى الامديد-السمبلاوين, السنبلاوين, الدقه...",اخري
242,"مركز نبروه, مصر",اخري


In [13]:
df_f.to_csv(r'data.csv', index=False)

In [14]:
data = pd.read_csv('data.csv')

X = data['address_label'].values
Y = data['address_city'].apply(lambda x: 1 if x == 'القاهرة' else 0).values

In [17]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

maxlen = 20
padded_sequences = pad_sequences(sequences, maxlen=maxlen)

X_train, X_val, y_train, y_val = train_test_split(padded_sequences, Y, test_size=0.2, random_state=42)

word_index = tokenizer.word_index
topwords = 5000

model = Sequential()
model.add(Embedding(topwords, 64, input_length=maxlen))
model.add(GRU(64, dropout=0.5, recurrent_dropout=0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])


Epoch 1/50
7/7 [==============================] - 4s 216ms/step - loss: 0.6886 - accuracy: 0.5641 - val_loss: 0.6745 - val_accuracy: 0.6122
Epoch 2/50
7/7 [==============================] - 1s 122ms/step - loss: 0.6701 - accuracy: 0.6051 - val_loss: 0.6571 - val_accuracy: 0.6122
Epoch 3/50
7/7 [==============================] - 1s 141ms/step - loss: 0.6480 - accuracy: 0.6051 - val_loss: 0.6254 - val_accuracy: 0.6122
Epoch 4/50
7/7 [==============================] - 1s 151ms/step - loss: 0.6076 - accuracy: 0.6308 - val_loss: 0.5723 - val_accuracy: 0.7755
Epoch 5/50
7/7 [==============================] - 1s 111ms/step - loss: 0.5350 - accuracy: 0.6769 - val_loss: 0.4842 - val_accuracy: 0.7959
Epoch 6/50
7/7 [==============================] - 2s 229ms/step - loss: 0.4409 - accuracy: 0.8000 - val_loss: 0.3930 - val_accuracy: 0.8367
Epoch 7/50
7/7 [==============================] - 1s 168ms/step - loss: 0.3406 - accuracy: 0.8667 - val_loss: 0.3102 - val_accuracy: 0.9184
Epoch 8/50
7/7 [====

In [18]:
model.save('model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
with open('tokenizer.pickle', 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)

In [21]:
sequences = loaded_tokenizer.texts_to_sequences(['القاهرة - رمسيس - ش فتح الله'])
maxlen = 20
padded_sequences = pad_sequences(sequences, maxlen=maxlen)
model.predict(padded_sequences)

1/1 [==============================] - 1s 641ms/step


array([[0.983286]], dtype=float32)